# convolution neural network

In [3]:
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))


def cnn_model(X, w, w2, w3, w4, w_o, p_keep_conv, p_keep_hidden):
    
    # Conv2d
    # l1a shape=(?, 28, 28, 32)
    # padding='SAME' means output data's dimension is same as input image's. 
    l1a = tf.nn.relu( tf.nn.conv2d(X, w, strides=[1,1,1,1], padding='SAME') )
    
    # Max pooling 
    # l1 shape=(?, 14, 14, 32)
    l1 = tf.nn.max_pool( l1a, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME' )
    
    l1 = tf.nn.dropout(l1, p_keep_conv)
    
    # Conv2d
    # l2a shape=(?, 14, 14, 64)
    l2a = tf.nn.relu( tf.nn.conv2d(l1, w2, strides=[1,1,1,1], padding='SAME') )

    # l2 shape=(?, 7, 7, 64)
    l2 = tf.nn.max_pool( l2a, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME' )
    
    l2 = tf.nn.dropout(l2, p_keep_conv)
    
    # l3a shape=(?, 7, 7, 128)
    l3a = tf.nn.relu( tf.nn.conv2d(l2, w3, strides=[1, 1, 1, 1], padding='SAME') )
    
    # l3 shape=(?, 4, 4, 128)
    l3 = tf.nn.max_pool( l3a, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    # reshape to (?, 2048)
    l3 = tf.reshape(l3, [-1, w4.get_shape().as_list()[0]])   
    l3 = tf.nn.dropout(l3, p_keep_conv)
    
    # Fully connected neural network
    l4 = tf.nn.relu(tf.matmul(l3, w4))
    l4 = tf.nn.dropout(l4, p_keep_hidden)

    pyx = tf.matmul(l4, w_o)
    return pyx

In [4]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
import numpy as np

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

# Parameters
training_epochs = 15
batch_size = 100
display_step = 1
#learning_rate = 10
#learning_rate = 0.1
#learning_rate = 0.01
learning_rate = 0.001
decay = 0.9

# MNIST data input (img shape: 28*28)
n_input = 784

# MNIST total classes (0-9 digits)
n_classes = 10

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [5]:
X = tf.placeholder("float", [None, 28, 28, 1])
Y = tf.placeholder("float", [None, 10])

trX = mnist.train.images
trY = mnist.train.labels
teX = mnist.test.images
teY = mnist.test.labels

trX = trX.reshape(-1, 28, 28, 1)  # 28x28x1 input img
teX = teX.reshape(-1, 28, 28, 1)  # 28x28x1 input img

w = init_weights([3, 3, 1, 32])       # 3x3x1 conv, 32 outputs
w2 = init_weights([3, 3, 32, 64])     # 3x3x32 conv, 64 outputs
w3 = init_weights([3, 3, 64, 128])    # 3x3x32 conv, 128 outputs
w4 = init_weights([128 * 4 * 4, 625]) # FC 128 * 4 * 4 inputs, 625 outputs
w_o = init_weights([625, 10])         # FC 625 inputs, 10 outputs (labels)

p_keep_conv = tf.placeholder("float")
p_keep_hidden = tf.placeholder("float")
py_x = cnn_model(X, w, w2, w3, w4, w_o, p_keep_conv, p_keep_hidden)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(py_x, Y))

# using RMSPropOptimizer
train_op = tf.train.RMSPropOptimizer(learning_rate, decay).minimize(cost)
predict_op = tf.argmax(py_x, 1)

init = tf.global_variables_initializer()

print "training starts"

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    # sess.run(init)
    
    for i in range(10):
        training_batch = zip( range(0, len(trX), batch_size), range(batch_size, len(trX), batch_size) )
        print training_batch[-1]
        
        for start, end in training_batch:
            if start % 10000 == 0:
                print start, end
            sess.run(train_op, feed_dict={X:trX[start:end], Y:trY[start:end], p_keep_conv:0.8, p_keep_hidden:0.5})
        
        test_indices = np.arange(len(teX)) 
        np.random.shuffle(test_indices)
        test_indices = test_indices[0:256]
        
        print "step: ", i , ", Accuracy: ", np.mean(np.argmax(teY[test_indices], axis=1) == sess.run(predict_op, feed_dict={X: teX[test_indices], Y: teY[test_indices], p_keep_conv: 1.0, p_keep_hidden: 1.0}))
        
    print "Optimization finished."

training starts
(54800, 54900)
0 100
10000 10100
20000 20100
30000 30100
40000 40100
50000 50100
step:  0 , Accuracy:  0.96484375
(54800, 54900)
0 100
10000 10100
20000 20100
30000 30100
40000 40100
50000 50100
step:  1 , Accuracy:  0.9921875
(54800, 54900)
0 100
10000 10100
20000 20100
30000 30100
40000 40100
50000 50100
step:  2 , Accuracy:  0.984375
(54800, 54900)
0 100
10000 10100
20000 20100
30000 30100
40000 40100
50000 50100
step:  3 , Accuracy:  1.0
(54800, 54900)
0 100
10000 10100
20000 20100
30000 30100
40000 40100
50000 50100
step:  4 , Accuracy:  0.97265625
(54800, 54900)
0 100
10000 10100
20000 20100
30000 30100
40000 40100
50000 50100
step:  5 , Accuracy:  0.98828125
(54800, 54900)
0 100
10000 10100
20000 20100
30000 30100
40000 40100
50000 50100
step:  6 , Accuracy:  0.99609375
(54800, 54900)
0 100
10000 10100
20000 20100
30000 30100
40000 40100
50000 50100
step:  7 , Accuracy:  0.99609375
(54800, 54900)
0 100
10000 10100
20000 20100
30000 30100
40000 40100
50000 50100
s